## Key SVC Parts Monitoring Dashboard

###### **Project Goal**:
There are over 30,000 active SVC parts used in LG Electronics' customer service operations. Each GBU has a designated PIC (Person in Charge) responsible for managing their own PSI (Production, Sales, and Inventory) to monitor and forecast parts demand trends and inventory levels.

This monitoring dashboard is designed to provide upper management with a clear overview of key parts inventory status and detailed insights, enabling better decision-making and proactive risk management.

In [1]:
# Importing Libraries

import pandas as pd
import numpy as np
import gcsfs
from datetime import datetime
from datetime import date, timedelta
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', None)

### Raw Data is collected from PIC every month for update

- Rep Part, Category, Division, Monthly Demand(6M), RA History (Return Authorization), Supply Delay Status
- Master Sheet for Rep Part as rep_part_df

In [2]:
folder_path = 'gs://keyparts_monitoring/'
file_name = 'Monitoring Full List.xlsx'

file_path = f'{folder_path}{file_name}'

fs = gcsfs.GCSFileSystem()
parts_list_df = pd.read_excel(fs.open(file_path), sheet_name='Imp List', usecols="A:K")
rep_part_df = pd.read_excel(fs.open(file_path), sheet_name='All 운영품번', usecols="A:B")

In [3]:
parts_list_df.head(2)

,Rep Part,Category,Div,10,11,12,1,2,3,RA 이력 (Y/N),공급지연 (Y/N)
0,4681EA2001T,[WM] Motor,WM,13525,7028,8815,12537,7717,8467,N,Y
1,AGM30045804,[WM] Parts Assembly,WM,1291,1469,1328,2077,1234,1896,Y,Y


In [4]:
parts_list_df[parts_list_df['Rep Part'] == 'EBR42005101']

,Rep Part,Category,Div,10,11,12,1,2,3,RA 이력 (Y/N),공급지연 (Y/N)
221,EBR42005101,[Cooking] PCB,Cooking,154,108,145,140,100,86,N,N


In [5]:
print(f'Total Number of Parts: ', parts_list_df['Rep Part'].nunique())

Total Number of Parts:  329


### Parts Inventory Data
- On Hand Inventory Quantity
- In transit Quantity

In [6]:
folder_path = 'gs://keyparts_monitoring/'
file_name = '4. Original Part Inventory.xlsx'

file_path = f'{folder_path}{file_name}'

fs = gcsfs.GCSFileSystem()
inventory_df = pd.read_excel(fs.open(file_path), sheet_name = 'Original Part Master', header=1, usecols="A:Q")

In [7]:
inventory_df.head(2)

,Original Part,Description,Part Status,Division Name,Inventory,ETA,bo_asc,bo_dms,bo_other,Comment,transit,open,OP AIR,OP TRK,OP SEA,TTL BO,RA
0,AGF80300704,"Filter Assembly,Water,Package Assembly,C",Active,REF,229834,5/16_23040 ; 5/19_57600 ; 6/04_23040 ; 6/09_38...,0,0,0,NaN,176400,118800,0,0,118800,0,0
1,ADQ73334008,"Filter Assembly,Air Cleaner",Active,REF,147565,5/13_607 ; 5/14_14872 ; 5/16_6289 ; 5/19_2776 ...,0,0,0,NaN,80552,117966,0,0,117966,0,0


- Key Parts List selected by PIC must be registered as Rep part in Rep Master sheet

In [8]:
inventory_df2 = inventory_df[['Original Part', 'Description', 'Inventory', 'transit', 'open']]
inventory_df2.head(2)

,Original Part,Description,Inventory,transit,open
0,AGF80300704,"Filter Assembly,Water,Package Assembly,C",229834,176400,118800
1,ADQ73334008,"Filter Assembly,Air Cleaner",147565,80552,117966


In [9]:
inventory_df2.shape

(259324, 5)

In [10]:
folder_path = 'gs://scm_dataset/Daily BO data/'
file_name = '20250509 - Backorder and Hold Report.xlsx'

file_path = f'{folder_path}{file_name}'

fs = gcsfs.GCSFileSystem()
df_kevin_awaiting = pd.read_excel(fs.open(file_path), sheet_name='Awaiting Shipping Details', header=1, usecols="B:AR")

In [11]:
df_kevin_awaiting.head(2)

,WH,Order Status Code,Order No*,SO Line No*,Delivery No,Request No,Repair Receipt No,Order Date,Days Since Order Date,Awaiting Shipping Reason,Picking Release Date,Picking Release Timestamp,Aging Groups,Shipping Compliance Date,Shipping Compliance,Air/Ground,Shipping Method,Category,Hold Type,Ship Set Name,SO Type,Item Status,Parts No,Item Description,Parts Class Code,Key Parts Category,Part Functionality,Customer Type with FOC,Ship To Code,Ship To Customer Type,Ship To Name,Bill To Code,Bill To Full Name,Company Code,Division Name,Division Code,Sealed System Part Type,Master State,COGO Amount (USD),Order Amount (USD),Order Count,Order Qty,Order/COGO Amount (Awaiting Shipping)
0,ESC,AWAITING_SHIPPING,713687939,2.1,NaN,PON240705820932,NaN,2024-07-05,308,Back Order Hold,NaT,NaT,8. 60+ days,2024-07-08,Late,Ground,TRUCK-TL-TL Standard,Ground (Other),BACK_ORDER_HOLD,NaN,Regular_OMV_AI,Active,EAJ65658001,"LCD,Module-TFT",EAJ0202,N,FUNCTIONAL,ASC,20754200,Repair SVC Center,"ADVISORY T.V. & RADIO LABS, L.L.C.",US050966001B,ADVISORY TELE,BS,MNT,GTT,NaN,NJ,461.822166,1086.6,1,2,1086.6
1,ESC,AWAITING_SHIPPING,713703838,1.1,NaN,PON240710839141,NaN,2024-07-10,303,Back Order Hold,NaT,NaT,8. 60+ days,2024-07-11,Late,Ground,TRUCK-TL-TL Standard,Ground (Other),BACK_ORDER_HOLD,NaN,Regular_OMV_AI,Active,EAJ65658001,"LCD,Module-TFT",EAJ0202,N,FUNCTIONAL,ASC,20754200,Repair SVC Center,"ADVISORY T.V. & RADIO LABS, L.L.C.",US050966001B,ADVISORY TELE,BS,MNT,GTT,NaN,NJ,461.822166,1086.6,1,2,1086.6


In [12]:
# Getting active items which are held as back order

df_kevin_awaiting2 = df_kevin_awaiting[[
    'Order Date', 'Days Since Order Date',
    'Awaiting Shipping Reason', 'Picking Release Date', 'Item Status',
    'Parts No', 'Parts Class Code','Item Description', 'Key Parts Category', 'Part Functionality',
    'Customer Type with FOC', 'Company Code', 'Division Code', 'Order Qty'
]].copy()

df_kevin_awaiting3 = df_kevin_awaiting2[(df_kevin_awaiting2['Awaiting Shipping Reason'] == 'Back Order Hold') & (df_kevin_awaiting2['Picking Release Date'].isna()) & (df_kevin_awaiting2['Item Status'] == 'Active')]

# deletion on unnecessary columns
df_kevin_awaiting3 = df_kevin_awaiting3.drop(columns=['Awaiting Shipping Reason', 'Picking Release Date', 'Item Status'])

# column name change
df_kevin_awaiting3 = df_kevin_awaiting3.rename(columns={
    'Days Since Order Date': 'Delay',
    'Item Description': 'Desc',
    'Part Functionality': 'Functionality',
    'Customer Type with FOC': 'Customer',
    'Key Parts Category': 'Key Parts',
    'Order Qty': 'BO qty'
})

In [13]:
df_kevin_awaiting3.head(5)

,Order Date,Delay,Parts No,Parts Class Code,Desc,Key Parts,Functionality,Customer,Company Code,Division Code,BO qty
0,2024-07-05,308,EAJ65658001,EAJ0202,"LCD,Module-TFT",N,FUNCTIONAL,ASC,BS,GTT,2
1,2024-07-10,303,EAJ65658001,EAJ0202,"LCD,Module-TFT",N,FUNCTIONAL,ASC,BS,GTT,2
2,2024-07-26,287,EAJ65658001,EAJ0202,"LCD,Module-TFT",N,FUNCTIONAL,ASC,BS,GTT,1
3,2024-08-01,281,EAJ65658001,EAJ0202,"LCD,Module-TFT",N,FUNCTIONAL,ASC,BS,GTT,1
5,2024-10-18,203,EAJ65804401,EAJ0202,"LCD,Module-TFT",[LTV] Module,FUNCTIONAL,ASC,HE,GLT,1


In [14]:
bo_info_df = df_kevin_awaiting3.groupby('Parts No').agg({
    'BO qty': 'sum'
}).reset_index()

In [15]:
bo_info_df.head(5)

,Parts No,BO qty
0,0IPMGKE053B,1
1,1TPL0302818,2
2,2B71165R,2
3,3040EA1005A,1
4,3210EL1003A,1


In [16]:
bo_merge = pd.merge(inventory_df2, bo_info_df, left_on='Original Part', right_on='Parts No', how='left')
bo_merge.drop(columns=['Parts No'], inplace=True)
bo_merge['BO qty'] = pd.to_numeric(bo_merge['BO qty'], errors='coerce').fillna(0).astype(int)

In [17]:
bo_merge.head(2)

,Original Part,Description,Inventory,transit,open,BO qty
0,AGF80300704,"Filter Assembly,Water,Package Assembly,C",229834,176400,118800,0
1,ADQ73334008,"Filter Assembly,Air Cleaner",147565,80552,117966,0


In [18]:
inventory_merge = pd.merge(bo_merge, rep_part_df, left_on='Original Part', right_on='Part#', how='inner')

In [19]:
inventory_merge.head(2)

,Original Part,Description,Inventory,transit,open,BO qty,Part#,Rep Part#
0,AGF80300704,"Filter Assembly,Water,Package Assembly,C",229834,176400,118800,0,AGF80300704,AGF80300704
1,ADQ73334008,"Filter Assembly,Air Cleaner",147565,80552,117966,0,ADQ73334008,ADQ73334008


In [20]:
inventory_new = inventory_merge.groupby('Rep Part#').agg({
    'Inventory': 'sum',
    'transit': 'sum',
    'open': 'sum',
    'BO qty': 'sum',
    'Description': 'first'
}).reset_index()

#### Inventory and Transit quantity information is aggregated by Rep Part, which we can now merge it to our original raw data

In [21]:
inventory_new.sample(n=10)

,Rep Part#,Inventory,transit,open,BO qty,Description
34298,AKC74070201,6,0,5,0,"Bucket Assembly,Ice"
42696,EBZ37191902,0,0,0,0,"TBD,Other Outsourcing SVC"
26724,AEC73438105,12,0,4,0,"Guide Assembly,Rail"
15448,6411W1A032G,0,0,0,0,Power Cord Assembly
19942,ABQ76660203,4,0,0,0,"Case Assembly,Lamp"
5050,3650W1A128F,0,0,0,0,"Handle,Door"
3619,3552W0A006A,0,0,0,0,"Cover,Choke"
51872,MHL62131405,4,0,0,0,Shelf
19185,ABQ59260013,1,0,0,0,"Case Assembly,Control(Indoor)"
47864,MDQ63318201,1,0,0,0,"Frame,Funnel"


In [22]:
print(f'shape: {inventory_new.shape}')

shape: (53432, 6)


In [23]:
df1 = pd.merge(parts_list_df, inventory_new, left_on='Rep Part', right_on='Rep Part#', how='left')
df1 = df1.drop(columns=['Rep Part#'])

In [24]:
parts_list_df[parts_list_df['Rep Part'] == '4400EL2001F']

,Rep Part,Category,Div,10,11,12,1,2,3,RA 이력 (Y/N),공급지연 (Y/N)
2,4400EL2001F,[WM] Others,WM,1519,962,1496,1469,1374,1441,N,N


In [25]:
inventory_new[inventory_new['Rep Part#'] == '4400EL2001F']

,Rep Part#,Inventory,transit,open,BO qty,Description
9628,4400EL2001F,1834,29,2850,0,"Belt,Poly V"


In [26]:
# Validity Check if every merging is correcly operated

df1[df1['Inventory'].isnull()]

,Rep Part,Category,Div,10,11,12,1,2,3,RA 이력 (Y/N),공급지연 (Y/N),Inventory,transit,open,BO qty,Description


In [27]:
df1.head(3)

,Rep Part,Category,Div,10,11,12,1,2,3,RA 이력 (Y/N),공급지연 (Y/N),Inventory,transit,open,BO qty,Description
0,4681EA2001T,[WM] Motor,WM,13525,7028,8815,12537,7717,8467,N,Y,22857,6865,29877,0,"Motor Assembly,AC,Pump"
1,AGM30045804,[WM] Parts Assembly,WM,1291,1469,1328,2077,1234,1896,Y,Y,2568,1957,2569,0,"Thermostat,Parts Assembly,SVC"
2,4400EL2001F,[WM] Others,WM,1519,962,1496,1469,1374,1441,N,N,1834,29,2850,0,"Belt,Poly V"


In [28]:
# Confirmed that the original row number matches with the row number of output

df1.shape

(329, 16)

#### Adding Supplier information to the dataframe

In [29]:
folder_path = 'gs://keyparts_monitoring/'
file_name = 'Supplier Master 2025 0328.xlsx'

file_path = f'{folder_path}{file_name}'

supplier_df = pd.read_excel(fs.open(file_path), header=1, sheet_name='Today', usecols="E:G")

In [30]:
supplier_df.head(2)

,Part No,Supplier,Supplier Code
0,49113005,ENAI_PT. LG ELECTRONICS INDONESIA_ID000016,ID000016
1,53402025,ENAI_LG ELECTRONICS INC._KR044335,KR044335


In [31]:
supplier_df_selected = supplier_df[['Part No', 'Supplier Code']]

df2 = pd.merge(df1, supplier_df_selected, left_on='Rep Part', right_on='Part No', how='inner')
df2 = df2.drop(columns=['Part No'])

In [32]:
df2

,Rep Part,Category,Div,10,11,12,1,2,3,RA 이력 (Y/N),공급지연 (Y/N),Inventory,transit,open,BO qty,Description,Supplier Code
0,4681EA2001T,[WM] Motor,WM,13525,7028,8815,12537,7717,8467,N,Y,22857,6865,29877,0,"Motor Assembly,AC,Pump",KR044335
1,AGM30045804,[WM] Parts Assembly,WM,1291,1469,1328,2077,1234,1896,Y,Y,2568,1957,2569,0,"Thermostat,Parts Assembly,SVC",KR044335
2,4400EL2001F,[WM] Others,WM,1519,962,1496,1469,1374,1441,N,N,1834,29,2850,0,"Belt,Poly V",US000491
3,4561EL3002A,[WM] Motor,WM,934,535,832,988,818,684,N,N,2961,87,1098,0,"Pulley Assembly,Motor",KR044335
4,5301EL1001H,[WM] Heater,WM,283,261,337,598,452,406,N,N,492,0,0,0,Heater Assembly,KR044335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,EBR77384108,[CAC] PCB,AC,38,16,27,42,35,63,N,N,178,291,93,0,"PCB Assembly,Main",KR044335
324,EBR79004805,[CAC] PCB,AC,36,33,33,26,29,28,N,N,92,71,45,0,"PCB Assembly,Main",KR044335
325,ADP35994403,[CAC] FAN,AC,46,9,38,29,21,24,N,N,135,127,32,0,"Fan Assembly,Cross Flow",KR044335
326,EBG61107301,[CAC] THERMISTOR,AC,17,22,43,28,28,25,N,N,190,56,44,0,"Thermistor Assembly,NTC",KR044335


In [33]:
df2.columns

Index([     'Rep Part',      'Category',           'Div',              10,
                    11,              12,               1,               2,
                     3,   'RA 이력 (Y/N)',    '공급지연 (Y/N)',     'Inventory',
             'transit',          'open',        'BO qty',   'Description',
       'Supplier Code'],
      dtype='object')

### Calculating the Current T/O which determines if PIC needs to place or cancel the order
- (Inventory + transit) / (3 months average) will be the formula to calculate the current turnover rate
- Code must be modified every month for the update

In [34]:
df2.columns = df2.columns.map(str)  # converting every column into string data type

recent_avg = (df2['1'] + df2['2'] + df2['3']) / 3

df2['Current T/O'] = np.where(recent_avg != 0, round((df2['Inventory'] + df2['transit'] - df2['BO qty']) / recent_avg, 2), 0)
df2.columns = [
    'rep_part', 'category', 'division',
    'm10', 'm11', 'm12', 'm1', 'm2', 'm3',
    'ra_history_yn', 'sd_yn', 'inventory',
    'transit', 'open', 'BO qty', 'description',
    'supplier_code', 'current_TO']

##### Demand Surge or Demand Drop notification function

In [35]:
early_avg = (df2['m10'] + df2['m11'] + df2['m12']) / 3
recent_avg = (df2['m1'] + df2['m2'] + df2['m3']) / 3

df2['demand_trend'] = np.where(
    recent_avg >= early_avg * 1.3, 'Surge',
    np.where(recent_avg <= early_avg * 0.7, 'Drop', 'Normal'))

In [36]:
df2.head(2)

,rep_part,category,division,m10,m11,m12,m1,m2,m3,ra_history_yn,sd_yn,inventory,transit,open,BO qty,description,supplier_code,current_TO,demand_trend
0,4681EA2001T,[WM] Motor,WM,13525,7028,8815,12537,7717,8467,N,Y,22857,6865,29877,0,"Motor Assembly,AC,Pump",KR044335,3.10,Normal
1,AGM30045804,[WM] Parts Assembly,WM,1291,1469,1328,2077,1234,1896,Y,Y,2568,1957,2569,0,"Thermostat,Parts Assembly,SVC",KR044335,2.61,Normal


In [37]:
df2['demand_trend'].value_counts()

,count
demand_trend,
Normal,192
Surge,104
Drop,32


#### Add bulky part information by importing detailed bulky data, and set different alert turnover thresholds for bulky (2.5) and non-bulky (2) parts to notify PICs only when their attention is required (Review column).

In [38]:
folder_path = 'gs://keyparts_monitoring/'
file_name = 'Original Part Inventory 2025 0428.xlsx'

file_path = f'{folder_path}{file_name}'

bulky_info_df = pd.read_excel(fs.open(file_path), sheet_name='Original Part Master', header=1, usecols="A,V")

In [39]:
bulky_info_df

,Original Part,Y/N
0,AGF80300704,N
1,AGF80232402,N
2,ADQ73334008,N
3,AGF80300812,N
4,AGF80300702,N
...,...,...
259126,ZEN1350,N
259127,ZEN1355,N
259128,ZEN1360,N
259129,ZEN300,N


In [40]:
bulky_added = pd.merge(df2, bulky_info_df, left_on='rep_part', right_on='Original Part', how='inner')
bulky_added = bulky_added.drop('Original Part', axis=1)

In [41]:
bulky_added

,rep_part,category,division,m10,m11,m12,m1,m2,m3,ra_history_yn,sd_yn,inventory,transit,open,BO qty,description,supplier_code,current_TO,demand_trend,Y/N
0,4681EA2001T,[WM] Motor,WM,13525,7028,8815,12537,7717,8467,N,Y,22857,6865,29877,0,"Motor Assembly,AC,Pump",KR044335,3.10,Normal,N
1,AGM30045804,[WM] Parts Assembly,WM,1291,1469,1328,2077,1234,1896,Y,Y,2568,1957,2569,0,"Thermostat,Parts Assembly,SVC",KR044335,2.61,Normal,N
2,4400EL2001F,[WM] Others,WM,1519,962,1496,1469,1374,1441,N,N,1834,29,2850,0,"Belt,Poly V",US000491,1.30,Normal,N
3,4561EL3002A,[WM] Motor,WM,934,535,832,988,818,684,N,N,2961,87,1098,0,"Pulley Assembly,Motor",KR044335,3.67,Normal,N
4,5301EL1001H,[WM] Heater,WM,283,261,337,598,452,406,N,N,492,0,0,0,Heater Assembly,KR044335,1.01,Surge,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,EBR77384108,[CAC] PCB,AC,38,16,27,42,35,63,N,N,178,291,93,0,"PCB Assembly,Main",KR044335,10.05,Surge,N
324,EBR79004805,[CAC] PCB,AC,36,33,33,26,29,28,N,N,92,71,45,0,"PCB Assembly,Main",KR044335,5.89,Normal,N
325,ADP35994403,[CAC] FAN,AC,46,9,38,29,21,24,N,N,135,127,32,0,"Fan Assembly,Cross Flow",KR044335,10.62,Normal,N
326,EBG61107301,[CAC] THERMISTOR,AC,17,22,43,28,28,25,N,N,190,56,44,0,"Thermistor Assembly,NTC",KR044335,9.11,Normal,N


In [42]:
bulky_added['Review'] = np.where(
    bulky_added['Y/N'] == 'N', np.where(bulky_added['current_TO'] < 2, 'Yes', 'No'),
    np.where(bulky_added['current_TO'] < 2.5, 'Yes', 'No'))

In [43]:
bulky_added.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   rep_part       328 non-null    object 
 1   category       328 non-null    object 
 2   division       328 non-null    object 
 3   m10            328 non-null    int64  
 4   m11            328 non-null    int64  
 5   m12            328 non-null    int64  
 6   m1             328 non-null    int64  
 7   m2             328 non-null    int64  
 8   m3             328 non-null    int64  
 9   ra_history_yn  328 non-null    object 
 10  sd_yn          328 non-null    object 
 11  inventory      328 non-null    int64  
 12  transit        328 non-null    int64  
 13  open           328 non-null    int64  
 14  BO qty         328 non-null    int64  
 15  description    328 non-null    object 
 16  supplier_code  328 non-null    object 
 17  current_TO     328 non-null    float64
 18  demand_tre

In [44]:
bulky_added

,rep_part,category,division,m10,m11,m12,m1,m2,m3,ra_history_yn,sd_yn,inventory,transit,open,BO qty,description,supplier_code,current_TO,demand_trend,Y/N,Review
0,4681EA2001T,[WM] Motor,WM,13525,7028,8815,12537,7717,8467,N,Y,22857,6865,29877,0,"Motor Assembly,AC,Pump",KR044335,3.10,Normal,N,No
1,AGM30045804,[WM] Parts Assembly,WM,1291,1469,1328,2077,1234,1896,Y,Y,2568,1957,2569,0,"Thermostat,Parts Assembly,SVC",KR044335,2.61,Normal,N,No
2,4400EL2001F,[WM] Others,WM,1519,962,1496,1469,1374,1441,N,N,1834,29,2850,0,"Belt,Poly V",US000491,1.30,Normal,N,Yes
3,4561EL3002A,[WM] Motor,WM,934,535,832,988,818,684,N,N,2961,87,1098,0,"Pulley Assembly,Motor",KR044335,3.67,Normal,N,No
4,5301EL1001H,[WM] Heater,WM,283,261,337,598,452,406,N,N,492,0,0,0,Heater Assembly,KR044335,1.01,Surge,N,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,EBR77384108,[CAC] PCB,AC,38,16,27,42,35,63,N,N,178,291,93,0,"PCB Assembly,Main",KR044335,10.05,Surge,N,No
324,EBR79004805,[CAC] PCB,AC,36,33,33,26,29,28,N,N,92,71,45,0,"PCB Assembly,Main",KR044335,5.89,Normal,N,No
325,ADP35994403,[CAC] FAN,AC,46,9,38,29,21,24,N,N,135,127,32,0,"Fan Assembly,Cross Flow",KR044335,10.62,Normal,N,No
326,EBG61107301,[CAC] THERMISTOR,AC,17,22,43,28,28,25,N,N,190,56,44,0,"Thermistor Assembly,NTC",KR044335,9.11,Normal,N,No


In [45]:
bulky_added.rename(columns={'Y/N': 'bulky'}, inplace=True)

In [46]:
bulky_added['Review'].value_counts()

,count
Review,
No,293
Yes,35


In [47]:
bulky_added[bulky_added['rep_part'] == '3751DD1001J']

,rep_part,category,division,m10,m11,m12,m1,m2,m3,ra_history_yn,sd_yn,inventory,transit,open,BO qty,description,supplier_code,current_TO,demand_trend,bulky,Review


In [48]:
bulky_added.columns

Index(['rep_part', 'category', 'division', 'm10', 'm11', 'm12', 'm1', 'm2',
       'm3', 'ra_history_yn', 'sd_yn', 'inventory', 'transit', 'open',
       'BO qty', 'description', 'supplier_code', 'current_TO', 'demand_trend',
       'bulky', 'Review'],
      dtype='object')

In [49]:
demand_cols = ['m10', 'm11', 'm12', 'm1', 'm2', 'm3']

# melt로 월별 수요 데이터를 long 형태로 변환
df_melted = bulky_added.melt(
    id_vars=['rep_part', 'category', 'division',
       'ra_history_yn', 'sd_yn', 'inventory', 'transit', 'open',
       'BO qty', 'description', 'supplier_code', 'current_TO', 'demand_trend',
       'bulky', 'Review'],
    value_vars=demand_cols,
    var_name='month',
    value_name='demand'
)

In [50]:
def convert_month(m):
    month_num = int(m[1:])  # 'm9' → 9
    if month_num >= 9:
        return f"2024-{month_num:02d}"  # 작년 하반기
    else:
        return f"2025-{month_num:02d}"  # 올해 상반기

df_melted['month'] = df_melted['month'].apply(convert_month)
print(df_melted[['rep_part', 'month', 'demand']].head(12))

       rep_part    month  demand
0   4681EA2001T  2024-10   13525
1   AGM30045804  2024-10    1291
2   4400EL2001F  2024-10    1519
3   4561EL3002A  2024-10     934
4   5301EL1001H  2024-10     283
5   4681EL1008A  2024-10     283
6   EAU64283201  2024-10     429
7   5220FR2008F  2024-10     166
8   4560EL3001A  2024-10     234
9   EBR31002611  2024-10     245
10  EBR31002601  2024-10     183
11  3045EL1002K  2024-10     144


In [51]:
df_melted[df_melted['rep_part'] == '4681EA2001T']

,rep_part,category,division,ra_history_yn,sd_yn,inventory,transit,open,BO qty,description,supplier_code,current_TO,demand_trend,bulky,Review,month,demand
0,4681EA2001T,[WM] Motor,WM,N,Y,22857,6865,29877,0,"Motor Assembly,AC,Pump",KR044335,3.1,Normal,N,No,2024-10,13525
328,4681EA2001T,[WM] Motor,WM,N,Y,22857,6865,29877,0,"Motor Assembly,AC,Pump",KR044335,3.1,Normal,N,No,2024-11,7028
656,4681EA2001T,[WM] Motor,WM,N,Y,22857,6865,29877,0,"Motor Assembly,AC,Pump",KR044335,3.1,Normal,N,No,2024-12,8815
984,4681EA2001T,[WM] Motor,WM,N,Y,22857,6865,29877,0,"Motor Assembly,AC,Pump",KR044335,3.1,Normal,N,No,2025-01,12537
1312,4681EA2001T,[WM] Motor,WM,N,Y,22857,6865,29877,0,"Motor Assembly,AC,Pump",KR044335,3.1,Normal,N,No,2025-02,7717
1640,4681EA2001T,[WM] Motor,WM,N,Y,22857,6865,29877,0,"Motor Assembly,AC,Pump",KR044335,3.1,Normal,N,No,2025-03,8467


In [52]:
df_melted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968 entries, 0 to 1967
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   rep_part       1968 non-null   object 
 1   category       1968 non-null   object 
 2   division       1968 non-null   object 
 3   ra_history_yn  1968 non-null   object 
 4   sd_yn          1968 non-null   object 
 5   inventory      1968 non-null   int64  
 6   transit        1968 non-null   int64  
 7   open           1968 non-null   int64  
 8   BO qty         1968 non-null   int64  
 9   description    1968 non-null   object 
 10  supplier_code  1968 non-null   object 
 11  current_TO     1968 non-null   float64
 12  demand_trend   1968 non-null   object 
 13  bulky          1968 non-null   object 
 14  Review         1968 non-null   object 
 15  month          1968 non-null   object 
 16  demand         1968 non-null   int64  
dtypes: float64(1), int64(5), object(11)
memory usage: 26

In [53]:
from google.cloud import storage

client = storage.Client()

bucket_name = 'keyparts_monitoring'
file_name = 'lgeai-scm-a539d1607964.json'

bucket = client.get_bucket(bucket_name)
blob = bucket.blob(file_name)
blob.download_to_filename(r'C:\Users\hyoju.kang\Downloads\lgeai-scm-1dcb486c8275.json')

In [54]:
from google.cloud import storage

client = storage.Client.from_service_account_json(r'C:\Users\hyoju.kang\Downloads\lgeai-scm-1dcb486c8275.json')

bucket_name = 'keyparts_monitoring'
bucket = client.get_bucket(bucket_name)

In [55]:
final_df = df_melted

storage_client = client = storage.Client.from_service_account_json(r'C:\Users\hyoju.kang\Downloads\lgeai-scm-1dcb486c8275.json')

bucket_name = 'keyparts_monitoring'
blob_name = 'final_df.csv'  # file name

bucket = storage_client.get_bucket(bucket_name)
final_df.to_csv('/tmp/final_df.csv', index=False)

blob = bucket.blob(blob_name)
blob.upload_from_filename('/tmp/final_df.csv')

In [56]:
from google.cloud import bigquery

client = bigquery.Client.from_service_account_json(r'C:\Users\hyoju.kang\Downloads\lgeai-scm-1dcb486c8275.json')

project_id = 'lgeai-scm'
dataset_id = 'monitoring_data'

full_dataset_id = f"{project_id}.{dataset_id}"

try:
    client.delete_dataset(full_dataset_id, not_found_ok=True)
except Exception as e:
    print(f"⚠️ 데이터셋 삭제 중 오류: {e}")

dataset = bigquery.Dataset(full_dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)

print(f"✅ 데이터셋 '{dataset_id}' 새로 생성 완료")

⚠️ 데이터셋 삭제 중 오류: 400 DELETE https://bigquery.googleapis.com/bigquery/v2/projects/lgeai-scm/datasets/monitoring_data?prettyPrint=false: Dataset lgeai-scm:monitoring_data is still in use
✅ 데이터셋 'monitoring_data' 새로 생성 완료


In [57]:
from google.cloud import bigquery

client = bigquery.Client.from_service_account_json(r'C:\Users\hyoju.kang\Downloads\lgeai-scm-1dcb486c8275.json')

uri = "gs://keyparts_monitoring/final_df.csv"

dataset_id = 'monitoring_data'
table_id = f'{dataset_id}.final_df_cleaned'

job_config = bigquery.LoadJobConfig(
    skip_leading_rows=1,
    source_format=bigquery.SourceFormat.CSV,
    autodetect=True,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)

load_job = client.load_table_from_uri(
    uri,
    table_id,
    job_config=job_config,
)

load_job.result()

print(f"데이터가 BigQuery 테이블 '{table_id}'에 성공적으로 로드되었습니다.")

데이터가 BigQuery 테이블 'monitoring_data.final_df_cleaned'에 성공적으로 로드되었습니다.
